## CORACLE-VAR

### Data Processing Step for Confounders

In [133]:
## Ignore warnings
import warnings
warnings.filterwarnings("ignore")
# Helper imports
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal # For NYSE trading calendar
import os
import sys

from itertools import product
from functools import reduce
import matplotlib.pyplot as plt

%matplotlib inline

current_working_dir = os.getcwd()
print(f"Current Working Directory: {current_working_dir}")
project_root = os.path.dirname(current_working_dir)
modules_path = os.path.join(project_root, 'Modules')
if modules_path not in sys.path:
    sys.path.append(modules_path)
    print(f"Added to sys.path for custom modules: {modules_path}")

####################################################################
#### NYSE Daily Open-Close Returns
####################################################################
data_folder_path = os.path.join(project_root, 'Data')
data_file_name = "OPCL_20000103_20201231.csv"   
data_file_path = os.path.join(data_folder_path, data_file_name) # So that we get to the file itself and not the folder it is in
returns_df = pd.read_csv(data_file_path) # Assumes file exists and is readable
returns_df.set_index('ticker', inplace=True)
returns_df.columns = pd.to_datetime(returns_df.columns.str.lstrip('X'), format='%Y%m%d').strftime('%Y-%m-%d')
returns_df_cleaned = returns_df.dropna().transpose() # Assumes dropna results in non-empty returns_df
returns_df_cleaned.index = pd.to_datetime(returns_df_cleaned.index)
returns_df_cleaned.index.name = 'date'
print("Data loaded and cleaned. Sample (first 5 rows/cols):")
print(returns_df_cleaned.iloc[0:5,0:5])
print(f"Shape of the cleaned data: {returns_df_cleaned.shape}")

####################################################################
#### Constructing the dataframe for the Confounding Variables
####################################################################

"""
Data Source: Federal Reserve Economic Data (FRED)

DFF -> Federal Funds Effective Rate
T5YIE -> 5-Year Breakeven Inflation Rate (only from 2003)
USEPUINDXD -> Economic Policy Uncertainty Index for United States; News-based, 7 days moving average
VIX -> CBOE Volatility Index (VIX)
DCOILWTICO -> West Texas Intermediate (WTI) Crude Oil Prices: Cushing, Oklahoma
DTWEXBGS -> Broad U.S. Dollar Index: Trade Weighted Exchange Rate Index for Major Currencies (only from 2006)
DTWEXEMEGS-> Broad U.S. Dollar Index: Trade Weighted Exchange Rate Index for Emerging Market Economies (only from 2006)

Other Potential Data Sources:
Gold Prices (Futures/Spot), etc.
"""

files = ["DFF_20000103_20201231.csv",
         "T5YIE_20030102_20201231.csv",
         "USEPUINDXD_20000103_20201231.csv",
         "VIX_20000103_20201231.csv",
         "DCOILWTICO_20000103_20201231.csv",
         "DTWEXBGS_20060102_20201231.csv",
         "DTWEXEMEGS_20060102_20201231.csv"
         ]


merged_confound_df = reduce(
    lambda left, right: pd.merge(left, right, on="observation_date", how="outer"),
    (pd.read_csv(os.path.join(data_folder_path, f), parse_dates=[0]) for f in files)
)
imputed_confound_df = merged_confound_df.interpolate(method='linear', limit_direction='both')
imputed_confound_df.set_index('observation_date', inplace=True)
imputed_confound_df.index.name = 'date'  # Renaming index to 'date'
print("Confounding Variables DataFrame constructed and cleaned. Sample (first 5 rows/cols):")
print(imputed_confound_df.iloc[0:5,:])
print(f"Shape of the cleaned confounding variables data: {imputed_confound_df.shape}")
filtered_confound_df = imputed_confound_df[imputed_confound_df.index.isin(returns_df_cleaned.index)]
print("Filtered Confounding Variables DataFrame:")
print(filtered_confound_df.head())
print(f"Shape of the filtered confounding variables data: {filtered_confound_df.shape}")

Current Working Directory: c:\Users\hktan\OneDrive - University of California\Codes\ICAIF_25\New Code\Script
Data loaded and cleaned. Sample (first 5 rows/cols):
ticker            AA       ABM       ABT       ADI       ADM
date                                                        
2000-01-03 -0.013042 -0.009188 -0.007117 -0.036071  0.000000
2000-01-04  0.010043  0.012346 -0.012786 -0.044261  0.005277
2000-01-05  0.047628 -0.006192  0.011111  0.014493 -0.015915
2000-01-06 -0.011713  0.000000  0.032553 -0.027719  0.010695
2000-01-07 -0.016118  0.003091  0.028573  0.033654  0.005249
Shape of the cleaned data: (5279, 663)
Confounding Variables DataFrame constructed and cleaned. Sample (first 5 rows/cols):
             DFF  T5YIE  USEPUINDXD  VIXCLS  DCOILWTICO  DTWEXBGS  DTWEXEMEGS
date                                                                         
2000-01-01  3.99    1.3       68.04   24.21       25.56  101.4155    100.9386
2000-01-02  3.99    1.3      119.36   24.21       25.

### Applying DML for orthogonalization

Remarks on installing econml to run the code with the correct dependencies will be included

In [134]:
## Clustering packages etc
import seaborn as sns
from parallelized_runs import run_sliding_window_var_evaluation_vectorized
import multiprocessing

warnings.filterwarnings("ignore", category=UserWarning, module='statsmodels')
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

try:
    from signet.cluster import Cluster
except ImportError:
    print("Signet package not found. Attempting to install from GitHub...")
    try:
        import subprocess
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "git+https://github.com/alan-turing-institute/SigNet.git"]
        )
        # This part of the code should go first since importing parallelized_runs already requires the signet package
        from signet.cluster import Cluster
        print("Signet package installed successfully.")
    except Exception as e:
        print(f"Error installing Signet package: {e}")
        print("Please install it manually: pip install git+https://github.com/alan-turing-institute/SigNet.git")


from econml.dml import LinearDML, SparseLinearDML
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.multioutput import MultiOutputRegressor 
from sklearn.base import clone
%matplotlib inline

### OR-VAR(p)

We first look at a version of this without the ACLE (adaptive causal lag esimation) portion. This assumes that lag values are fixed, and we correct for confounding and fit for the relevant nuisance functions directly.

In [168]:
p = 4                           # how many lags
def make_lags(df, p):
    return pd.concat([df.shift(k).add_suffix(f'_lag{k}') for k in range(1, p+1)], axis=1)

#Y_df = returns_df_cleaned.iloc[:, :20]
Y_df = returns_df_cleaned.iloc[:, :20]
T_df = make_lags(Y_df, p)
W_df = make_lags(filtered_confound_df, p)
#W_df = make_lags(filtered_confound_df,p)

full = pd.concat([Y_df, T_df, W_df], axis=1).dropna()   # drop the first p rows with NaNs
train_size = int(len(full)*0.7)

train, test = full.iloc[:train_size], full.iloc[train_size:]
print(train.index[[0,-1]])   # sanity‑check the split dates
print(test.index[[0,-1]])


# --- 2. align everything & turn into np.arrays ----------------------------
Y_cols = Y_df.columns
T_cols = T_df.columns
W_cols = W_df.columns

def to_arrays(df):
    return df[Y_cols].values, df[T_cols].values, df[W_cols].values

Y_tr, T_tr, W_tr = to_arrays(train)
Y_te, T_te, W_te = to_arrays(test)

# --- 3. fit Double ML ---------------------------------------------------
tscv = TimeSeriesSplit(5)          # same blocked CV inside train only


fast_tree = ExtraTreesRegressor(
    n_estimators=100,
    max_depth=5,
    min_samples_split=20,
    random_state=0
)

est = LinearDML(model_y=fast_tree,
                model_t=fast_tree,        
                cv=tscv,
                discrete_treatment=False,
                random_state=0)

est.fit(Y_tr, T_tr, X=None, W=W_tr)

# --- 4. evaluate the model - CORRECTED --------------
T_next = T_te[0, :].reshape(1, -1)
W_next = W_te[0, :].reshape(1, -1)

# The structure is: est.models_y[0] contains the 5 CV fold models
Y_base_folds = []
for model in est.models_y[0]:  # Note: iterate through est.models_y[0], not est.models_y
    pred = model.predict(W_next)  # Shape: (1, 20)
    Y_base_folds.append(pred)

# Convert to array and average across folds
Y_base_array = np.array(Y_base_folds)  # Shape: (5, 1, 20)
print(f"Y_base_array shape: {Y_base_array.shape}")

# Average across folds (axis=0)
Y_base = np.mean(Y_base_array, axis=0)  # Shape: (1, 20)
print(f"Y_base shape after averaging: {Y_base.shape}")

# Get treatment effects
theta = est.const_marginal_ate()  # Shape: (20, 60)
print(f"Theta shape: {theta.shape}")

# Compute prediction
Y_hat_next = Y_base + T_next @ theta.T
print(f"Y_hat_next shape: {Y_hat_next.shape}")
print("1‑step‑ahead forecast:", Y_hat_next.flatten()[:5], "...")

# Compare with realized returns
Y_true_next = Y_te[0, :]
rmse_next = root_mean_squared_error(Y_true_next, Y_hat_next.flatten())
print(f"RMSE for 1-step-ahead point: {rmse_next:.4f}")

DatetimeIndex(['2000-01-07', '2014-09-17'], dtype='datetime64[ns]', name='date', freq=None)
DatetimeIndex(['2014-09-18', '2020-12-31'], dtype='datetime64[ns]', name='date', freq=None)
Y_base_array shape: (5, 1, 20)
Y_base shape after averaging: (1, 20)
Theta shape: (20, 80)
Y_hat_next shape: (1, 20)
1‑step‑ahead forecast: [0.   0.   0.   0.   0.01] ...
RMSE for 1-step-ahead point: 0.0135


In [92]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from econml.dml import LinearDML

def make_lags(df, p):
    """Create lagged variables for p periods"""
    return pd.concat([df.shift(k).add_suffix(f'_lag{k}') for k in range(1, p+1)], axis=1)

Y_df = returns_df_cleaned.iloc[:, :20]
T_df = make_lags(Y_df, p)
W_df = filtered_confound_df

# --- 1. Wrapper for MultiOutput regressors ----------------------------
def get_regressor(regressor_name, force_multioutput=True, **kwargs):
    """Factory function to create different regressors with MultiOutput wrapper"""
    base_regressors = {
        'extra_trees': ExtraTreesRegressor(
            n_estimators=kwargs.get('n_estimators', 100),
            max_depth=kwargs.get('max_depth', 5),
            min_samples_split=kwargs.get('min_samples_split', 20),
            random_state=kwargs.get('random_state', 0)
        ),
        'random_forest': RandomForestRegressor(
            n_estimators=kwargs.get('n_estimators', 100),
            max_depth=kwargs.get('max_depth', 5),
            min_samples_split=kwargs.get('min_samples_split', 20),
            random_state=kwargs.get('random_state', 0)
        ),
    }
    
    base_model = base_regressors[regressor_name]
    
    # For Y model (multiple outputs), we might need MultiOutputRegressor
    if force_multioutput:
        return MultiOutputRegressor(base_model)
    else:
        return base_model

# --- 2. Corrected prediction function --------------------------------
def predict_double_ml_full_with_lags(Y_df, W_df, 
                                    p=3,  # number of lags
                                    model_y_name='extra_trees', 
                                    model_t_name='extra_trees',
                                    model_y_params=None,
                                    model_t_params=None,
                                    cv_folds=5,
                                    train_ratio=0.7):
    
    if model_y_params is None:
        model_y_params = {}
    if model_t_params is None:
        model_t_params = {}
    
    # CREATE LAGGED TREATMENT VARIABLES
    T_df = make_lags(Y_df, p)  # This creates lagged versions of Y as treatments
    W_df = make_lags(W_df, p)  # Assuming W_df also needs lags
    
    # Create full dataset and drop NAs (first p rows will have NAs due to lags)
    full = pd.concat([Y_df, T_df, W_df], axis=1).dropna()
    train_size = int(len(full) * train_ratio)
    
    train, test = full.iloc[:train_size], full.iloc[train_size:]
    
    # Get column names
    Y_cols = Y_df.columns  # 20 assets
    T_cols = T_df.columns  # 20 assets × p lags
    W_cols = W_df.columns  # confounders
    
    print(f"Y shape: {len(Y_cols)}, T shape: {len(T_cols)}, W shape: {len(W_cols)}")
    print(f"Number of lags: {p}")
    print(f"T_cols sample: {T_cols[:5].tolist()}")  # Show first 5 treatment columns
    
    def to_arrays(df):
        return df[Y_cols].values, df[T_cols].values, df[W_cols].values
    
    Y_tr, T_tr, W_tr = to_arrays(train)
    Y_te, T_te, W_te = to_arrays(test)
    
    print(f"Training shapes - Y: {Y_tr.shape}, T: {T_tr.shape}, W: {W_tr.shape}")
    print(f"Test shapes - Y: {Y_te.shape}, T: {T_te.shape}, W: {W_te.shape}")
    
    # Create regressors
    model_y = get_regressor(model_y_name, force_multioutput=False, **model_y_params)
    model_t = get_regressor(model_t_name, force_multioutput=False, **model_t_params)
    
    # Set up cross-validation
    tscv = TimeSeriesSplit(cv_folds)
    
    # Fit DoubleML
    est = LinearDML(
        model_y=model_y,
        model_t=model_t,
        cv=tscv,
        discrete_treatment=False,
        random_state=0
    )
    
    print("Fitting DoubleML model...")
    est.fit(Y_tr, T_tr, X=None, W=W_tr)
    
    # Get treatment effects and check dimensions
    theta = est.const_marginal_ate()
    print(f"Treatment effects shape: {theta.shape}")
    
    # Make predictions
    predictions = corrected_predict(est, T_te, W_te, Y_te.shape)
    
    # Calculate MSE
    mse_per_asset = np.mean((Y_te - predictions) ** 2, axis=0)
    overall_rmse = np.sqrt(np.mean(mse_per_asset))
    
    results = {
        'rmse_overall': overall_rmse,
        'mse_per_asset': mse_per_asset,
        'predictions': predictions,
        'actuals': Y_te,
        'model': est,
        'theta_shape': theta.shape,
        'asset_names': Y_cols,
        'treatment_names': T_cols,
        'n_lags': p
    }
    
    return results

# --- Modified rolling window function with lags -------------------
def rolling_window_evaluation_lookback_with_lags(Y_df, W_df,
                                                p=3,  # number of lags
                                                window_size=20,
                                                lookback_days=252,
                                                model_y_name='extra_trees',
                                                model_t_name='extra_trees',
                                                model_y_params=None,
                                                model_t_params=None,
                                                cv_folds=5,
                                                train_ratio=0.7,
                                                verbose=True):
    """
    Rolling window evaluation with proper lagged treatment variables
    """
    
    if model_y_params is None:
        model_y_params = {}
    if model_t_params is None:
        model_t_params = {}
    
    # CREATE LAGGED TREATMENT VARIABLES
    T_df = make_lags(Y_df, p)
    
    # Create full dataset and drop NAs
    full = pd.concat([Y_df, T_df, W_df], axis=1).dropna()
    
    # Adjust for the fact that we lose p rows due to lagging
    if train_ratio is None:
        initial_train_size = lookback_days
    else:
        initial_train_size = int(max(train_ratio*full.shape[0], lookback_days))
    
    # Get column names
    Y_cols = Y_df.columns
    T_cols = T_df.columns
    W_cols = W_df.columns
    
    if verbose:
        print(f"Created {len(T_cols)} treatment variables from {p} lags of {len(Y_cols)} assets")
        print(f"Sample treatment columns: {T_cols[:5].tolist()}")
    
    # Storage for predictions and actuals
    all_predictions = []
    all_actuals = []
    prediction_dates = []
    training_times = []
    
    # Start from initial_train_size and go through each day
    test_start = initial_train_size
    test_end = len(full)
    
    if verbose:
        print(f"Starting rolling window evaluation with {p} lags...")
        print(f"Lookback window: {lookback_days} days")
        print(f"Initial training size: {initial_train_size}")
        print(f"Test period: {test_end - test_start} days")
        print(f"RMSE window size: {window_size} days")
    
    import time
    
    for current_day in range(test_start, test_end):
        if verbose and (current_day - test_start) % 10 == 0:
            progress = (current_day - test_start) / (test_end - test_start) * 100
            print(f"Processing day {current_day - test_start + 1}/{test_end - test_start} ({progress:.1f}%)")
        
        start_time = time.time()
        
        # Use only the last lookback_days for training
        train_start_idx = max(0, current_day - lookback_days)
        train_data = full.iloc[train_start_idx:current_day]
        test_data = full.iloc[current_day:current_day+1]  # Just one day
        
        # Extract arrays
        Y_tr = train_data[Y_cols].values
        T_tr = train_data[T_cols].values
        W_tr = train_data[W_cols].values
        
        Y_te = test_data[Y_cols].values[0]  # Get 1D array for single day
        T_te = test_data[T_cols].values[0]
        W_te = test_data[W_cols].values[0]
        
        # Create and fit model
        model_y = get_regressor(model_y_name, force_multioutput=False, **model_y_params)
        model_t = get_regressor(model_t_name, force_multioutput=False, **model_t_params)
        
        tscv = TimeSeriesSplit(cv_folds)
        
        est = LinearDML(
            model_y=model_y,
            model_t=model_t,
            cv=tscv,
            discrete_treatment=False,
            random_state=0
        )
        
        # Fit model
        est.fit(Y_tr, T_tr, X=None, W=W_tr)
        
        # Make prediction for current day
        prediction = corrected_predict_single_day(est, T_te, W_te)
        
        # Store results
        all_predictions.append(prediction)
        all_actuals.append(Y_te)
        prediction_dates.append(full.index[current_day])
        
        # Track timing
        training_times.append(time.time() - start_time)
    
    # Convert to arrays
    all_predictions = np.array(all_predictions)
    all_actuals = np.array(all_actuals)
    
    # Calculate rolling window RMSE
    window_rmses = []
    window_dates = []
    per_asset_window_rmses = []
    
    for i in range(len(all_predictions) - window_size + 1):
        window_pred = all_predictions[i:i+window_size]
        window_actual = all_actuals[i:i+window_size]
        
        # Calculate RMSE for this window
        mse = np.mean((window_actual - window_pred) ** 2)
        rmse = np.sqrt(mse)
        
        # Also calculate per-asset RMSE for this window
        mse_per_asset = np.mean((window_actual - window_pred) ** 2, axis=0)
        rmse_per_asset = np.sqrt(mse_per_asset)
        
        window_rmses.append(rmse)
        per_asset_window_rmses.append(rmse_per_asset)
        window_dates.append((prediction_dates[i], prediction_dates[i+window_size-1]))
    
    # Calculate overall metrics
    overall_mse_per_asset = np.mean((all_actuals - all_predictions) ** 2, axis=0)
    overall_rmse_per_asset = np.sqrt(overall_mse_per_asset)
    overall_mse = np.mean((all_actuals - all_predictions) ** 2)
    overall_rmse = np.sqrt(overall_mse)
    
    results = {
        'predictions': all_predictions,
        'actuals': all_actuals,
        'prediction_dates': prediction_dates,
        'window_rmses': np.array(window_rmses),
        'per_asset_window_rmses': np.array(per_asset_window_rmses),
        'window_dates': window_dates,
        'rmse_per_asset': overall_rmse_per_asset,
        'overall_rmse': overall_rmse,
        'asset_names': Y_cols,
        'treatment_names': T_cols,
        'window_size': window_size,
        'lookback_days': lookback_days,
        'n_lags': p,
        'avg_training_time': np.mean(training_times),
        'total_training_time': np.sum(training_times)
    }
    
    if verbose:
        print(f"\nEvaluation complete!")
        print(f"Overall RMSE: {overall_rmse:.6f}")
        print(f"Average window RMSE: {np.mean(window_rmses):.6f}")
        print(f"Min window RMSE: {np.min(window_rmses):.6f}")
        print(f"Max window RMSE: {np.max(window_rmses):.6f}")
        print(f"Average training time per day: {np.mean(training_times):.2f} seconds")
        print(f"Total training time: {np.sum(training_times)/60:.2f} minutes")
    
    return results

# --- 3. Corrected prediction logic -----------------------------------
def corrected_predict(est, T_te, W_te, Y_shape):
    """Make predictions with proper dimension handling"""
    n_test, n_assets = Y_shape
    
    predictions = np.zeros((n_test, n_assets))
    
    # Get treatment effects
    theta = est.const_marginal_ate()
    print(f"Inside predict - theta shape: {theta.shape}")
    
    for t in range(n_test):
        T_t = T_te[t, :].reshape(1, -1)  # Shape: (1, 90)
        W_t = W_te[t, :].reshape(1, -1)  # Shape: (1, n_confounders)
        
        # Get baseline prediction
        Y_base_folds = []
        for model in est.models_y[0]:
            pred = model.predict(W_t)  # Should be shape (1, 20)
            Y_base_folds.append(pred)
        
        Y_base = np.mean(Y_base_folds, axis=0)  # Shape: (1, 20)
        
        treatment_effect = T_t @ theta.T  # (1, 20p) @ (20p, 20) = (1, 20)
        
        # Compute final prediction
        Y_hat_t = Y_base + treatment_effect  # Both should be (1, 20)
        predictions[t, :] = Y_hat_t.flatten()
        
        if t == 0:  # Debug first prediction
            print(f"First prediction shapes:")
            print(f"  Y_base: {Y_base.shape}")
            print(f"  treatment_effect: {treatment_effect.shape}")
            print(f"  Y_hat_t: {Y_hat_t.shape}")
    
    return predictions

def corrected_predict_single_day(est, T_t, W_t):
    """Make prediction for a single day - more efficient version"""
    # Ensure inputs are 2D
    T_t = T_t.reshape(1, -1) if T_t.ndim == 1 else T_t  # Shape: (1, n_features)
    W_t = W_t.reshape(1, -1) if W_t.ndim == 1 else W_t  # Shape: (1, n_confounders)
    
    # Get treatment effects
    theta = est.const_marginal_ate()
    
    # Get baseline prediction by averaging across CV folds
    Y_base_folds = []
    for model in est.models_y[0]:
        pred = model.predict(W_t)  # Shape: (1, n_assets)
        Y_base_folds.append(pred)
    
    Y_base = np.mean(Y_base_folds, axis=0)  # Shape: (1, n_assets)
    
    # Calculate treatment effect
    treatment_effect = T_t @ theta.T  # (1, n_features) @ (n_features, n_assets) = (1, n_assets)
    
    # Final prediction
    Y_hat = Y_base + treatment_effect  # Both are (1, n_assets)
    
    return Y_hat.flatten()  # Return as 1D array of shape (n_assets,)




# --- 5. Usage examples -----------------------------------------------
if __name__ == "__main__":
    # Example usage with proper lags
    # Assuming you have Y_df (returns) and W_df (confounders) defined
    
    # Test with different numbers of lags
    for p in [1, 3, 5]:
        print(f"\n=== Testing with {p} lags ===")
        results = predict_double_ml_full_with_lags(
            Y_df, filtered_confound_df, 
            p=p,
            model_y_name='extra_trees',
            model_t_name='extra_trees',
            train_ratio=0.7
        )
        print(f"RMSE with {p} lags: {results['rmse_overall']:.6f}")
    
    # Rolling window evaluation with lags
    print(f"\n=== Rolling Window Evaluation with Lags ===")
    rolling_results = rolling_window_evaluation_lookback_with_lags(
        Y_df, filtered_confound_df,
        p=3,  # Use 3 lags
        window_size=20,
        lookback_days=252,
        train_ratio = 0.95,
    )
    print(f"Rolling window RMSE: {rolling_results['overall_rmse']:.6f}")


=== Testing with 1 lags ===
Y shape: 20, T shape: 20, W shape: 7
Number of lags: 1
T_cols sample: ['AA_lag1', 'ABM_lag1', 'ABT_lag1', 'ADI_lag1', 'ADM_lag1']
Training shapes - Y: (3694, 20), T: (3694, 20), W: (3694, 7)
Test shapes - Y: (1584, 20), T: (1584, 20), W: (1584, 7)
Fitting DoubleML model...
Treatment effects shape: (20, 20)
Inside predict - theta shape: (20, 20)
First prediction shapes:
  Y_base: (1, 20)
  treatment_effect: (1, 20)
  Y_hat_t: (1, 20)
RMSE with 1 lags: 0.017453

=== Testing with 3 lags ===
Y shape: 20, T shape: 60, W shape: 21
Number of lags: 3
T_cols sample: ['AA_lag1', 'ABM_lag1', 'ABT_lag1', 'ADI_lag1', 'ADM_lag1']
Training shapes - Y: (3693, 20), T: (3693, 60), W: (3693, 21)
Test shapes - Y: (1583, 20), T: (1583, 60), W: (1583, 21)
Fitting DoubleML model...
Treatment effects shape: (20, 60)
Inside predict - theta shape: (20, 60)
First prediction shapes:
  Y_base: (1, 20)
  treatment_effect: (1, 20)
  Y_hat_t: (1, 20)
RMSE with 3 lags: 0.017852

=== Testin

In [72]:
if __name__ == "__main__":
    print(f"\n=== Rolling Window Evaluation with Lags ===")
    rolling_results_2 = rolling_window_evaluation_lookback_with_lags(
        Y_df, W_df,
        p=1,  # Use 1 lags
        window_size=20,
        lookback_days=252,
        train_ratio = 0.95,
    )
    print(f"Rolling window RMSE: {rolling_results_2['overall_rmse']:.6f}")


=== Rolling Window Evaluation with Lags ===
Created 20 treatment variables from 1 lags of 20 assets
Sample treatment columns: ['AA_lag1', 'ABM_lag1', 'ABT_lag1', 'ADI_lag1', 'ADM_lag1']
Starting rolling window evaluation with 1 lags...
Lookback window: 252 days
Initial training size: 5014
Test period: 264 days
RMSE window size: 20 days
Processing day 1/264 (0.0%)
Processing day 11/264 (3.8%)
Processing day 21/264 (7.6%)
Processing day 31/264 (11.4%)
Processing day 41/264 (15.2%)
Processing day 51/264 (18.9%)
Processing day 61/264 (22.7%)
Processing day 71/264 (26.5%)
Processing day 81/264 (30.3%)
Processing day 91/264 (34.1%)
Processing day 101/264 (37.9%)
Processing day 111/264 (41.7%)
Processing day 121/264 (45.5%)
Processing day 131/264 (49.2%)
Processing day 141/264 (53.0%)
Processing day 151/264 (56.8%)
Processing day 161/264 (60.6%)
Processing day 171/264 (64.4%)
Processing day 181/264 (68.2%)
Processing day 191/264 (72.0%)
Processing day 201/264 (75.8%)
Processing day 211/264 (

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

# Set style for better-looking plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

def plot_rolling_rmse_detailed(results, figsize=(15, 10)):
    """Create a comprehensive visualization of rolling window RMSE results"""
    
    fig, axes = plt.subplots(2, 2, figsize=figsize)
    fig.suptitle(f'Rolling Window RMSE Analysis (Window Size: {results["window_size"]} days)', 
                 fontsize=16, y=1.02)
    
    # 1. Rolling RMSE over time
    ax1 = axes[0, 0]
    window_rmses = results['window_rmses']
    window_start_indices = range(len(window_rmses))
    
    ax1.plot(window_start_indices, window_rmses, 'b-', linewidth=2, alpha=0.7)
    ax1.axhline(y=results['overall_rmse'], color='r', linestyle='--', 
                linewidth=2, label=f'Overall RMSE: {results["overall_rmse"]:.6f}')
    ax1.axhline(y=np.mean(window_rmses), color='g', linestyle='--', 
                linewidth=2, label=f'Avg Window RMSE: {np.mean(window_rmses):.6f}')
    
    # Add confidence bands
    rmse_std = np.std(window_rmses)
    rmse_mean = np.mean(window_rmses)
    ax1.fill_between(window_start_indices, 
                     rmse_mean - rmse_std, 
                     rmse_mean + rmse_std, 
                     alpha=0.2, color='gray', 
                     label=f'±1 STD: {rmse_std:.6f}')
    
    ax1.set_xlabel('Window Number')
    ax1.set_ylabel('RMSE')
    ax1.set_title('Rolling Window RMSE Over Time')
    ax1.legend(loc='upper right')
    ax1.grid(True, alpha=0.3)
    
    # 2. RMSE Distribution
    ax2 = axes[0, 1]
    ax2.hist(window_rmses, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    ax2.axvline(x=np.mean(window_rmses), color='g', linestyle='--', linewidth=2)
    ax2.axvline(x=np.median(window_rmses), color='orange', linestyle='--', linewidth=2)
    ax2.set_xlabel('RMSE')
    ax2.set_ylabel('Frequency')
    ax2.set_title('Distribution of Window RMSEs')
    ax2.legend(['Mean', 'Median', 'Distribution'])
    
    # Add statistics text
    stats_text = f'Mean: {np.mean(window_rmses):.6f}\n'
    stats_text += f'Median: {np.median(window_rmses):.6f}\n'
    stats_text += f'Std: {np.std(window_rmses):.6f}\n'
    stats_text += f'Min: {np.min(window_rmses):.6f}\n'
    stats_text += f'Max: {np.max(window_rmses):.6f}'
    ax2.text(0.98, 0.97, stats_text, transform=ax2.transAxes, 
             verticalalignment='top', horizontalalignment='right',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # 3. Per-Asset RMSE
    ax3 = axes[1, 0]
    rmse_per_asset = results['rmse_per_asset']
    asset_names = results['asset_names']
    
    # Show top 10 assets by RMSE
    n_assets_to_show = min(10, len(asset_names))
    sorted_indices = np.argsort(rmse_per_asset)[::-1][:n_assets_to_show]
    
    y_pos = np.arange(n_assets_to_show)
    ax3.barh(y_pos, rmse_per_asset[sorted_indices], alpha=0.7)
    ax3.set_yticks(y_pos)
    ax3.set_yticklabels([asset_names[i] for i in sorted_indices])
    ax3.set_xlabel('RMSE')
    ax3.set_title(f'Top {n_assets_to_show} Assets by RMSE')
    ax3.grid(True, axis='x', alpha=0.3)
    
    # Add average line
    ax3.axvline(x=np.mean(rmse_per_asset), color='r', linestyle='--', 
                linewidth=2, label='Average')
    ax3.legend()
    
    # 4. Cumulative prediction error over time
    ax4 = axes[1, 1]
    predictions = results['predictions']
    actuals = results['actuals']
    
    # Calculate cumulative squared error
    squared_errors = (predictions - actuals) ** 2
    cumulative_mse = np.cumsum(np.mean(squared_errors, axis=1))
    cumulative_rmse = np.sqrt(cumulative_mse / np.arange(1, len(cumulative_mse) + 1))
    
    ax4.plot(cumulative_rmse, 'b-', linewidth=2)
    ax4.set_xlabel('Prediction Day')
    ax4.set_ylabel('Cumulative RMSE')
    ax4.set_title('Cumulative RMSE Over Test Period')
    ax4.grid(True, alpha=0.3)
    
    # Add final value annotation
    final_rmse = cumulative_rmse[-1]
    ax4.text(0.98, 0.02, f'Final: {final_rmse:.6f}', 
             transform=ax4.transAxes, 
             verticalalignment='bottom', 
             horizontalalignment='right',
             bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))
    
    plt.tight_layout()
    return fig

def plot_rmse_time_series(results, figsize=(14, 8)):
    """Plot RMSE time series with dates on x-axis"""
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=figsize, 
                                   gridspec_kw={'height_ratios': [2, 1]})
    
    # Extract data
    window_rmses = results['window_rmses']
    window_dates = results['window_dates']
    
    # Get start dates for each window
    window_start_dates = [dates[0] for dates in window_dates]
    
    # Convert to pandas for easier plotting
    rmse_series = pd.Series(window_rmses, index=pd.to_datetime(window_start_dates))
    
    # Plot 1: RMSE over time
    ax1.plot(rmse_series.index, rmse_series.values, 'b-', linewidth=2, label='Window RMSE')
    
    # Add moving average
    ma_window = 10
    rmse_ma = rmse_series.rolling(window=ma_window, center=True).mean()
    ax1.plot(rmse_ma.index, rmse_ma.values, 'r-', linewidth=2, 
             label=f'{ma_window}-window MA', alpha=0.8)
    
    # Add reference lines
    ax1.axhline(y=results['overall_rmse'], color='green', linestyle='--', 
                linewidth=1.5, label=f'Overall RMSE: {results["overall_rmse"]:.6f}')
    
    # Highlight high RMSE periods
    threshold = np.percentile(window_rmses, 90)
    high_rmse_mask = rmse_series > threshold
    if high_rmse_mask.any():
        ax1.fill_between(rmse_series.index, 0, 1, 
                        where=high_rmse_mask,
                        transform=ax1.get_xaxis_transform(),
                        alpha=0.2, color='red', 
                        label=f'Top 10% RMSE (>{threshold:.6f})')
    
    ax1.set_ylabel('RMSE')
    ax1.set_title(f'Rolling {results["window_size"]}-Day Window RMSE Over Time')
    ax1.legend(loc='upper right')
    ax1.grid(True, alpha=0.3)
    
    # Format x-axis
    ax1.xaxis.set_major_formatter(DateFormatter('%Y-%m-%d'))
    ax1.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    # Plot 2: Number of days with RMSE above/below average
    ax2.bar(rmse_series.index, 
           (rmse_series > rmse_series.mean()).astype(int),
           width=1, alpha=0.6, color='red', label='Above Average')
    ax2.bar(rmse_series.index, 
           (rmse_series <= rmse_series.mean()).astype(int) * -1,
           width=1, alpha=0.6, color='green', label='Below Average')
    
    ax2.set_ylabel('Above/Below\nAverage')
    ax2.set_xlabel('Date')
    ax2.legend(loc='upper right')
    ax2.grid(True, alpha=0.3)
    
    # Format x-axis
    ax2.xaxis.set_major_formatter(DateFormatter('%Y-%m-%d'))
    ax2.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    plt.tight_layout()
    return fig

def plot_asset_heatmap(results, figsize=(12, 8)):
    """Create a heatmap of per-asset RMSE over time windows"""
    
    per_asset_window_rmses = results.get('per_asset_window_rmses')
    if per_asset_window_rmses is None:
        print("No per-asset window RMSE data available")
        return None
    
    # Create DataFrame for heatmap
    asset_names = results['asset_names']
    n_windows = len(per_asset_window_rmses)
    
    # Reshape data for heatmap
    heatmap_data = pd.DataFrame(
        per_asset_window_rmses.T,  # Transpose to have assets as rows
        index=asset_names,
        columns=[f'W{i}' for i in range(n_windows)]
    )
    
    # Create figure
    fig, ax = plt.subplots(figsize=figsize)
    
    # Create heatmap
    sns.heatmap(heatmap_data, 
                cmap='YlOrRd',
                cbar_kws={'label': 'RMSE'},
                ax=ax,
                fmt='.6f',
                linewidths=0.5)
    
    ax.set_title('Per-Asset RMSE Across Time Windows')
    ax.set_xlabel('Window Number')
    ax.set_ylabel('Asset')
    
    # Rotate x-axis labels
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    # Show every nth label if too many windows
    if n_windows > 50:
        nth = n_windows // 20
        for i, label in enumerate(ax.xaxis.get_ticklabels()):
            if i % nth != 0:
                label.set_visible(False)
    
    plt.tight_layout()
    return fig

def plot_prediction_vs_actual_sample(results, n_assets_to_show=3, n_days_to_show=100):
    """Plot sample predictions vs actuals for selected assets"""
    
    predictions = results['predictions']
    actuals = results['actuals']
    asset_names = results['asset_names']
    dates = results['prediction_dates']
    
    # Select assets to show (best, median, worst by RMSE)
    rmse_per_asset = results['rmse_per_asset']
    sorted_indices = np.argsort(rmse_per_asset)
    
    assets_to_show = [
        sorted_indices[0],  # Best
        sorted_indices[len(sorted_indices)//2],  # Median
        sorted_indices[-1]  # Worst
    ]
    
    fig, axes = plt.subplots(n_assets_to_show, 1, figsize=(14, 4*n_assets_to_show))
    if n_assets_to_show == 1:
        axes = [axes]
    
    # Limit days to show
    end_idx = min(n_days_to_show, len(dates))
    date_range = dates[:end_idx]
    
    for idx, (ax, asset_idx) in enumerate(zip(axes, assets_to_show)):
        asset_name = asset_names[asset_idx]
        asset_rmse = rmse_per_asset[asset_idx]
        
        # Plot predictions and actuals
        ax.plot(date_range, actuals[:end_idx, asset_idx], 
                'b-', label='Actual', linewidth=2, alpha=0.7)
        ax.plot(date_range, predictions[:end_idx, asset_idx], 
                'r--', label='Predicted', linewidth=2, alpha=0.7)
        
        # Add error bands
        errors = predictions[:end_idx, asset_idx] - actuals[:end_idx, asset_idx]
        ax.fill_between(date_range, 
                       predictions[:end_idx, asset_idx] - np.std(errors),
                       predictions[:end_idx, asset_idx] + np.std(errors),
                       alpha=0.2, color='red', label='±1 STD Error')
        
        ax.set_title(f'{asset_name} - RMSE: {asset_rmse:.6f}')
        ax.set_ylabel('Value')
        ax.legend(loc='upper right')
        ax.grid(True, alpha=0.3)
        
        # Format x-axis
        ax.xaxis.set_major_formatter(DateFormatter('%Y-%m-%d'))
        plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    axes[-1].set_xlabel('Date')
    plt.tight_layout()
    return fig

# Example usage with the results from rolling window evaluation
if __name__ == "__main__":
    # Assuming you have results from the rolling window evaluation
    # results = rolling_window_evaluation_lookback(...)
    
    # Plot all visualizations
    fig1 = plot_rolling_rmse_detailed(rolling_results)
    plt.show()
    
    fig2 = plot_rmse_time_series(rolling_results)
    plt.show()
    
    fig3 = plot_asset_heatmap(rolling_results)
    plt.show()
    
    fig4 = plot_prediction_vs_actual_sample(rolling_results, n_assets_to_show=3)
    plt.show()
    
    # print("Plotting functions ready to use!")

### OR-VARACLE(p)

In [146]:
import scipy.stats as st
from scipy.stats import norm
from econml.inference import StatsModelsInference
from econml.sklearn_extensions.linear_model import StatsModelsLinearRegression
from statsmodels.stats.multitest import multipletests

In [135]:
Y_df = returns_df_cleaned.iloc[:, :20]
print(Y_df)

ticker            AA       ABM       ABT       ADI       ADM       ADX  \
date                                                                     
2000-01-03 -0.013042 -0.009188 -0.007117 -0.036071  0.000000 -0.001867   
2000-01-04  0.010043  0.012346 -0.012786 -0.044261  0.005277 -0.005666   
2000-01-05  0.047628 -0.006192  0.011111  0.014493 -0.015915  0.000000   
2000-01-06 -0.011713  0.000000  0.032553 -0.027719  0.010695  0.005742   
2000-01-07 -0.016118  0.003091  0.028573  0.033654  0.005249  0.003810   
...              ...       ...       ...       ...       ...       ...   
2020-12-24 -0.017156 -0.011338  0.008434  0.002495  0.000607 -0.000584   
2020-12-28  0.006315  0.009932 -0.010796 -0.011887 -0.002010  0.004078   
2020-12-29 -0.004527 -0.024098 -0.001568 -0.010525 -0.008056 -0.004066   
2020-12-30  0.039099 -0.002646 -0.002303  0.010522  0.009259  0.001745   
2020-12-31  0.002172  0.005565  0.012129  0.008361  0.007367  0.002896   

ticker           AEE       AEG       

In [ ]:
p_max = 5 # Maximum number of lags to consider

def auto_causal_lag_selection(p_max, Y_df, confound_df, 
                              model_y_name='extra_trees', 
                              model_t_name='extra_trees',
                              model_y_params=None,
                              model_t_params=None,
                              cv_folds=5):
    """
    Automatically select the best number of lags for causal inference
    using DML and OVB analysis.
    """
    if model_y_params is None:
        model_y_params = {}
    if model_t_params is None:
        model_t_params = {}

    n_assets = Y_df.shape[1]

    # Create lagged treatment variables
    T_df = Y_df.shift(1).add_suffix('_lag1')
    W_df = pd.concat([confound_df, confound_df.shift(1).add_suffix('_lag1')], axis=1)
        
    def realign(Y,T,W):
        full = pd.concat([Y, T, W], axis=1).dropna()
        Y_cols = Y.columns
        T_cols = T.columns
        W_cols = W.columns
        return full[Y_cols], full[T_cols], full[W_cols]

    lagged_assignments = {"t-1": "Treatment"}

    if p_max > 1:
        """ Alpha spending code 

        # We initialize the alpha-spending function for multiple-hypothesis testing downstream.
        # This is done via controlling the FWER error rate (FWER) using the alpha-spending method.
        # We do this via the O'Brien-Fleming approach, with each tests contributing "equally" to the total "information"
        # Initialize the alpha-spending function
        alpha_global = 0.05
        # equally spaced information fractions
        info = np.linspace(1/(p_max-1), 1, p_max-1)           
        info_prev = np.hstack(([0.0], info[:-1]))             

        def spend_OBF(t, alpha):
            z = norm.ppf(1 - alpha/2)
            return 2 * (1 - norm.cdf(z / np.sqrt(t)))
        
        # Calculate the alpha (significance level) for each test
        alpha_all = spend_OBF(info, alpha_global) - spend_OBF(info_prev, alpha_global)
        """ 

        alpha_all = [0.2]*(p_max-1)
        for i in range(len(alpha_all)):
            print(f"Alpha for lag {i+2}: {alpha_all[i]:.4f}")

        def create_index_mapping(T_names_prev, T_names_post, d_y):
            """
            Create index mapping as 2-tuples (pre_idx, post_idx) for coefficient comparison.
        
            Returns:
            - idx_pairs: List of (pre_idx, post_idx) tuples for existing coefficients
            - idx_new: List of post_idx for new coefficients
            """
            d_T_prev = len(T_names_prev)
            d_T_post = len(T_names_post)

            idx_pairs = []
            idx_new = []

            # Create mapping for all coefficients
            for y in range(d_y):
                for j_post, name_post in enumerate(T_names_post):
                    post_idx = y * d_T_post + j_post
                    
                    if name_post in T_names_prev:
                        # This is an existing coefficient
                        j_prev = T_names_prev.index(name_post)
                        pre_idx = y * d_T_prev + j_prev
                        idx_pairs.append((pre_idx, post_idx))
                    else:
                        # This is a new coefficient
                        idx_new.append(post_idx)
            
            return idx_pairs, idx_new


    for p in range(2, p_max + 1):
        
        #### Pre; ie p - 1
        Y_df, T_df, W_df = realign(Y_df, T_df, W_df)
        est_pre = LinearDML(
            model_y=get_regressor(model_y_name, force_multioutput=False, **model_y_params),
            model_t=get_regressor(model_t_name, force_multioutput=False, **model_t_params),
            cv=TimeSeriesSplit(n_splits=cv_folds),
            discrete_treatment=False,
            random_state=0
        )
        print(Y_df.shape, T_df.shape, W_df.shape)

        est_pre.fit(Y_df, T_df, X=None, W=W_df, inference=StatsModelsInference())
        est_pre_inf = est_pre.const_marginal_ate_inference()
        #print(est_pre_inf)
        theta_pre = est_pre_inf.mean_point.ravel()  # Flatten to 1-D vector
        cov_pre = est_pre_inf.mean_pred_stderr.ravel()

        #### Post; ie p
        T_df_temp = pd.concat([T_df, Y_df.shift(p).add_suffix(f'_lag{p}')], axis=1).dropna()
        W_df = pd.concat([W_df, confound_df.shift(p).add_suffix(f'_lag{p}')], axis=1).dropna()
        
        Y_df, T_df_temp, W_df = realign(Y_df, T_df_temp, W_df)

        est_post = LinearDML(
            model_y=get_regressor(model_y_name, force_multioutput=False, **model_y_params),
            model_t=get_regressor(model_t_name, force_multioutput=False, **model_t_params),
            cv=TimeSeriesSplit(cv_folds),
            discrete_treatment=False,
            random_state=0,
        )

        est_post.fit(Y_df, T_df_temp, X=None, W=W_df)
        est_post_inf = est_post.const_marginal_ate_inference()
        #print(est_post_inf)
        theta_post = est_post_inf.mean_point.ravel()  
        cov_post = est_post_inf.mean_pred_stderr.ravel()
        
        T_names_pre = T_df.columns.tolist()
        T_names_post = T_df_temp.columns.tolist()
        d_y   = Y_df.shape[1]                    # number of outcome series

        idx_pairs, idx_post = create_index_mapping(T_names_pre, T_names_post, d_y)

        # Within each test, we perform a Benjamini–Hochberg FDR test
        # p-values of signifinance
        p_sig_store = []
        for i in idx_post:
            z_stat = theta_post[i] / cov_post[i]
            p_value = 2 * (1 - norm.cdf(np.abs(z_stat)))
            p_sig_store.append(p_value)
        p_sig = min(multipletests(p_sig_store, alpha=0.05, method="fdr_bh")[1])
        print(p_sig)

        # p-value of drift without significance
        p_drift_store = []
        for i,j in idx_pairs:
            z_stat = (theta_post[j] - theta_pre[i])/ np.sqrt(cov_post[j]**2 + cov_pre[i]**2)
            p_value = 2 * (1 - norm.cdf(np.abs(z_stat)))
            p_drift_store.append(p_value)
        p_drift = min(multipletests(p_drift_store, alpha=0.05, method="fdr_bh")[1])
        print(p_drift)

        alpha = alpha_all[p-2]
        # Case 1: Significance of new coefficients
        if p_sig < alpha:
            print(f"✓ Significance detected for lag {p} (p-value: {p_sig:.4f})")
            lagged_assignments[f't-{p}'] = 'Treatment'
            T_df = T_df_temp
            continue

        # Case 2: Drift without Significance
        elif p_drift < alpha:
            print(f"✓ Drift detected for lag {p} (p-value: {p_drift:.4f})")
            lagged_assignments[f't-{p}'] = 'Confounding'
            W_df = pd.concat([W_df, Y_df.shift(p).add_suffix(f'_lag{p}')], axis=1).dropna()
            continue

        # Case 3: Neither
        else:
            print(f"✗ No significance or drift for lag {p} (p_sig: {p_sig:.4f}, p_drift: {p_drift:.4f})")
            p = p - 1
            break

    output = {
        'lagged_assignments': lagged_assignments,
        'final_lagged_treatment': T_df.columns.tolist(),
        'final_confounders': W_df.columns.tolist(),
        'W_df': W_df,
        'T_df': T_df,
        'Optimal p': p,
    }

    return output



print("Auto-causal lag selection example:")
print(auto_causal_lag_selection(
    p_max=5, 
    Y_df=Y_df, 
    confound_df=filtered_confound_df.iloc[-1000:,:],
    model_y_name='extra_trees',
    model_t_name='extra_trees',
    model_y_params={'n_estimators': 100},
    model_t_params={'n_estimators': 100},
    cv_folds=5
)['lagged_assignments'])


Auto-causal lag selection example:
Alpha for lag 2: 0.2000
Alpha for lag 3: 0.2000
Alpha for lag 4: 0.2000
Alpha for lag 5: 0.2000
(999, 20) (999, 20) (999, 14)
0.12154139808213671
0.9999957149887668
✓ Significance detected for lag 2 (p-value: 0.1215)
(997, 20) (997, 40) (997, 21)
0.0029005217773381275
0.9997929620968218
✓ Significance detected for lag 3 (p-value: 0.0029)
(994, 20) (994, 60) (994, 28)
0.06279661633682802
0.999775610096513
✓ Significance detected for lag 4 (p-value: 0.0628)
(990, 20) (990, 80) (990, 35)
0.3221580785496651
0.9998516660691872
✗ No significance or drift for lag 5 (p_sig: 0.3222, p_drift: 0.9999)
{'t-1': 'Treatment', 't-2': 'Treatment', 't-3': 'Treatment', 't-4': 'Treatment'}


In [176]:
from parallelized_runs import calculate_pnl

### Plotting Purposes (for Slides)

In [ ]:
p = 4                           # how many lags
def make_lags(df, p):
    return pd.concat([df.shift(k).add_suffix(f'_lag{k}') for k in range(1, p+1)], axis=1)

#Y_df = returns_df_cleaned.iloc[:, :20]
Y_df = returns_df_cleaned.iloc[:, :20]
T_df = make_lags(Y_df, p)
W_df = make_lags(filtered_confound_df, p)
#W_df = make_lags(filtered_confound_df,p)

full = pd.concat([Y_df, T_df, W_df], axis=1).dropna()   # drop the first p rows with NaNs
train_size = int(len(full)*0.7)
print(train_size, len(full))

train, test = full.iloc[train_size - 252*4:train_size], full.iloc[train_size:]
print(train.index[[0,-1]])   # sanity‑check the split dates
print(test.index[[0,-1]])


# --- 2. align everything & turn into np.arrays ----------------------------
Y_cols = Y_df.columns
T_cols = T_df.columns
W_cols = W_df.columns

def to_arrays(df):
    return df[Y_cols].values, df[T_cols].values, df[W_cols].values

Y_tr, T_tr, W_tr = to_arrays(train)
Y_te, T_te, W_te = to_arrays(test)

# --- 3. fit Double ML ---------------------------------------------------
tscv = TimeSeriesSplit(5)          # same blocked CV inside train only


fast_tree = ExtraTreesRegressor(
    n_estimators=100,
    max_depth=5,
    min_samples_split=20,
    random_state=0
)

#    fast_tree = RandomForestRegressor(
#        n_estimators=100,
#        max_depth=5,
#        min_samples_split=20,
#        random_state=0
#    )

est = LinearDML(model_y=fast_tree,
                model_t=fast_tree,        
                cv=tscv,
                discrete_treatment=False,
                random_state=0)

est.fit(Y_tr, T_tr, X=None, W=W_tr)

# --- 4. evaluate the model - CORRECTED --------------
T_next = T_te[0, :].reshape(1, -1)
W_next = W_te[0, :].reshape(1, -1)

# The structure is: est.models_y[0] contains the 5 CV fold models
Y_base_folds = []
for model in est.models_y[0]:  # Note: iterate through est.models_y[0], not est.models_y
    pred = model.predict(W_next)  # Shape: (1, 20)
    Y_base_folds.append(pred)

# Convert to array and average across folds
Y_base_array = np.array(Y_base_folds)  # Shape: (5, 1, 20)
print(f"Y_base_array shape: {Y_base_array.shape}")

# Average across folds (axis=0)
Y_base = np.mean(Y_base_array, axis=0)  # Shape: (1, 20)
print(f"Y_base shape after averaging: {Y_base.shape}")

# Get treatment effects
theta = est.const_marginal_ate()  # Shape: (20, 60)
print(f"Theta shape: {theta.shape}")

# Compute prediction
Y_hat_next = Y_base + T_next @ theta.T
print(f"Y_hat_next shape: {Y_hat_next.shape}")
print("1‑step‑ahead forecast:", Y_hat_next.flatten()[:5], "...")

# Compare with realized returns
Y_true_next = Y_te[0, :]
rmse_next = root_mean_squared_error(Y_true_next, Y_hat_next.flatten())
pnl_next = calculate_pnl(Y_hat_next, Y_true_next)
print(f"RMSE for 1-step-ahead point: {rmse_next:.4f}")

3692 5275
DatetimeIndex(['2010-09-15', '2014-09-17'], dtype='datetime64[ns]', name='date', freq=None)
DatetimeIndex(['2014-09-18', '2020-12-31'], dtype='datetime64[ns]', name='date', freq=None)
Y_base_array shape: (5, 1, 20)
Y_base shape after averaging: (1, 20)
Theta shape: (20, 80)
Y_hat_next shape: (1, 20)
1‑step‑ahead forecast: [0.   0.01 0.   0.01 0.01] ...


TypeError: calculate_pnl() got an unexpected keyword argument 'pnl_strategy'